In [142]:
import pandas as pd
import requests
import json
import matplotlib as mlt
import gmaps
from census import Census
from us import states
import numpy as np


census_API = 'ad9e020616792d8cbe0f666089dc3980b9d379e0'

c=Census(census_API)

#KCMO 311 Dataset
path = './Resources/311_Pothole_History.csv'

df = pd.DataFrame(pd.read_csv(path))
print(len(df))
clean_df = df.loc[(df['STATUS']=='RESOL')]
df.head(2)
clean_df = clean_df[['CASE ID', 'SOURCE', 'CREATION MONTH', 'CREATION YEAR', 'STATUS', 'EXCEEDED EST TIMEFRAME', 'DAYS TO CLOSE', 'ZIP CODE', 'NEIGHBORHOOD']]
clean_df.dropna(inplace=True)
clean_df['ZIP CODE'] = clean_df['ZIP CODE'].astype(int)
clean_df.head(5)
clean_df.to_csv('./Resources/PotholeData.csv', index=False)

50073


In [123]:
#Zip code list
zip_codes = df['ZIP CODE'].dropna().unique()
zip_codes = zip_codes.astype(int)
zip_codes

array([64133, 64131, 64130, 64119, 64134, 64108, 64113, 64132, 64112,
       64146, 64127, 64125, 64149, 64138, 64114, 64145, 64105, 64123,
       64110, 64137, 64155, 64109, 64154, 64151, 64124, 64116, 64120,
       64139, 64152, 64129, 64158, 64106, 64111, 64128, 64118, 64117,
       64136, 64126, 64157, 64156, 64101, 64163, 64153, 64161, 64147,
       64167, 64102, 64166, 64165, 64164, 64160, 64068, 64012, 64052,
       64030])

In [134]:
data = []
for ea in zip_codes:
    census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E","B19301_001E","B17001_002E"), {'for': 'zip code tabulation area:'+str(ea)})
    try:
        data.append(census_data[0])
    except:
        next

data = pd.DataFrame(data)
data.head()


,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,zip code tabulation area
0,ZCTA5 64133,50681.0,35163.0,40.8,26720.0,4690.0,64133
1,ZCTA5 64131,45688.0,22932.0,37.2,30411.0,4021.0,64131
2,ZCTA5 64130,31166.0,19996.0,39.3,21925.0,5238.0,64130
3,ZCTA5 64119,68841.0,27848.0,38.3,33224.0,1785.0,64119
4,ZCTA5 64134,41543.0,23874.0,31.3,20078.0,5507.0,64134


In [135]:
# Column Reordering
data = data.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})
data = data[['Zipcode','Household Income','Population','Median Age', 'Per Capita Income', 'Poverty Count']]
data = data.loc[(data['Population']!=0) & (data['Household Income']!=-666666666.0)]
data.head()

,Zipcode,Household Income,Population,Median Age,Per Capita Income,Poverty Count
0,64133,50681.0,35163.0,40.8,26720.0,4690.0
1,64131,45688.0,22932.0,37.2,30411.0,4021.0
2,64130,31166.0,19996.0,39.3,21925.0,5238.0
3,64119,68841.0,27848.0,38.3,33224.0,1785.0
4,64134,41543.0,23874.0,31.3,20078.0,5507.0


In [141]:
data.to_csv('./Resources/CensusData.csv', index=False)

In [145]:
ZipCodes = list(data['Zipcode'])
ZipCodes = pd.DataFrame(ZipCodes)
ZipCodes.to_csv('./Resources/Zipcodes.csv', index = False)